# 02-VGG19

一个图片分类的卷积神经网络。

In [ ]:
# Import here.
import torch
import collections
import path

import visdom

In [ ]:
# Visdom here.
g_is_visdom_on = True
try:
    g_vis = visdom.Visdom(raise_exceptions=True)
except ConnectionError:
    g_vis = None
    g_is_visdom_on = False

定义神经网络。

In [ ]:
import net
torch.manual_seed(3)
g_my_net = net.vgg19()

生成数据集。

In [ ]:
import dataset
_dir = path.Path('cifar-10-batches-py')
_data_batchs = [_dir / ('data_batch_%d' % i) for i in range(1, 5 + 1)]
_test_batchs = [_dir / 'test_batch']

g_train_dataset = dataset.make_dataset(_data_batchs)
g_test_dataset = dataset.make_dataset(_test_batchs)

初始化 visdom。

In [ ]:
g_win_loss = [None]
g_win_loss_options = {
    'title': 'Batch Loss Trace',
    'xlabel': 'Batch Count',
    'ylabel': 'Loss',
}

g_win_test = [None]
g_win_test_options = {
    'title': 'Test Loss Trace',
    'xlabel': 'Epoch Count',
    'ylabel': 'Loss',
}

初始化测试器。

In [ ]:
import test
def get_my_test():
    times = 0
    losses = []

    def my_test():
        nonlocal times
        nonlocal losses
        times = times + 1
        test.test(times, losses,
                  g_my_net,
                  torch.cuda.is_available(),
                  g_test_dataset,
                  100, 0, g_vis, g_win_test, g_win_test_options)

    return my_test

训练。

In [ ]:
import train
g_my_net.optimizer.state = collections.defaultdict(dict)
train.train(g_my_net, torch.cuda.is_available(), g_train_dataset,
            400, 0, 50, g_vis, g_win_loss, g_win_loss_options,
            on_test=get_my_test())